In [ ]:
"""
Cell For Papermill Parameters
"""

ARCH = 'resnet18'
imagesize = 100
NUM_CLASSES = 2
saved_model_name = "model_MRI.pth.tar"
data_dir="../../dataset/ds1/val/"
out_file = "uncertainty_MRI.csv"
wb_project = "Uncertainty_MRI"
WORKERS=4
GPU=0
SEED=1
VAL_BATCH = 6
forward_passes = 100

In [ ]:
import os
import random
import sys
import gc
import copy
import pickle

import numpy as np
import pandas as pd
import wandb
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [ ]:
with open('../config/config.json') as config_f:
    data = json.load(config_f)
aws_access_key_id = data['aws_access_key_id']
aws_secret_access_key = data['aws_secret_access_key']
region_name = data['region_name']
WANDB_API_KEY = data['WANDB_API_KEY']

In [ ]:
os.environ["WANDB_API_KEY"] = WANDB_API_KEY
wandb.login()
run = wandb.init(project=wb_project, entity='prostate-cancer', config={"imagesize":imagesize, "NUM_CLASSES":NUM_CLASSES})

wandb_run_name = wandb.run.name
wandb_run_id = wandb.run.id

config = wandb.config

config.ARCH  = ARCH 
config.imagesize  = imagesize 
config.forward_passes = forward_passes

### 2. Declare Functions Used for DNN Uncertainty

In [ ]:
def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        m.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.1, training=True))
                  
def get_model_monte_carlo_predictions(data_loader, forward_passes,
                                      model, n_classes, n_samples):
    """ Function to get the model monte-carlo samples and uncertainty estimates
    through multiple forward passes

    Parameters
    ----------
    data_loader : object
        data loader object from the data loader module
    forward_passes : int
        number of monte-carlo samples/forward passes
    model : object
        keras model
    n_classes : int
        number of classes in the dataset
    n_samples : int
        number of samples in the test set
    """

    dropout_predictions = np.empty((0, n_samples, n_classes))
    softmax = nn.Softmax(dim=1)
    
    model.eval()
    enable_dropout(model)
    
    for i in range(forward_passes):
        
        # counter
        if (((i+1) % 50) == 0) and ((i+1) >= 50):
            print("Number of passes finished:")
            print(i+1)
        
        predictions = np.empty((0, n_classes))

        for i, (image, label) in enumerate(data_loader):

            image = image.to(torch.device('cuda'))
            with torch.no_grad():
                output = model(image)
                output = softmax(output) # shape (n_samples, n_classes)
            predictions = np.vstack((predictions, output.cpu().numpy()))

        dropout_predictions = np.vstack((dropout_predictions,
                                         predictions[np.newaxis, :, :]))
        # dropout predictions - shape (forward_passes, n_samples, n_classes)
    
    # Calculating mean across multiple MCD forward passes 
    mean = np.mean(dropout_predictions, axis=0) # shape (n_samples, n_classes)

    # Calculating variance across multiple MCD forward passes 
    variance = np.var(dropout_predictions, axis=0) # shape (n_samples, n_classes)
    
    return tuple([mean, (variance**0.5)])

def get_input_monte_carlo_predictions(data_loader, forward_passes,
                                      model, n_classes, n_samples,
                                     num_channels, image_size):
    """ Function to get the monte-carlo samples and uncertainty estimates
    through multiple forward passes

    Parameters
    ----------
    data_loader : object
        data loader object from the data loader module
    forward_passes : int
        number of monte-carlo samples/forward passes
    model : object
        keras model
    n_classes : int
        number of classes in the dataset
    n_samples : int
        number of samples in the test set
    num_channels : int
        number of channels in the image
    image_size : int
        size of image in pixels, assuming a square image
    """
    
    noise_predictions = np.empty((0, n_samples, n_classes))
    softmax = nn.Softmax(dim=1)
    # create forward_passes number of monte carlo predictions for each unique image
    for i in range(forward_passes):
        
        # counter
        if (((i+1) % 50) == 0) and ((i+1) >= 50):
            print("Number of passes finished:")
            print(i+1)
        
        predictions = np.empty((0, n_classes))
        model.eval()

        for i, (image, label) in enumerate(data_loader):
            
            # adjust batchsize because the final loop may be less
            # than a full batch
            current_batchsize = image.shape[0]
            
            noise = torch.normal(mean = 0., std = 1., size = [current_batchsize, num_channels, 
                                                              image_size, image_size])

            noisy_image = image + noise

            # send images to gpu
            noisy_image = noisy_image.to(torch.device('cuda'))

            with torch.no_grad():
                output = model(noisy_image)
                output = softmax(output) # shape (n_samples, n_classes)
            predictions = np.vstack((predictions, output.cpu().numpy()))


        noise_predictions = np.vstack((noise_predictions,
                                         predictions[np.newaxis, :, :]))
        # noise predictions - shape (forward_passes, n_samples, n_classes)
    
    # Calculating mean across multiple forward passes 
    mean = np.mean(noise_predictions, axis=0) # shape (n_samples, n_classes)

    # Calculating variance across multiple forward passes 
    variance = np.var(noise_predictions, axis=0) # shape (n_samples, n_classes)
    
    return tuple([mean, (variance**0.5)])

### 3. Load DNN Model + Make Predictions / Uncertainty Estimates

#### 3.1 Load DNN MRI Model Parameters

#### 3.2 Load DNN Model

In [ ]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

torch.cuda.set_device(GPU)

model = models.__dict__[ARCH](pretrained = True)
inf = model.fc.in_features
model.fc = nn.Linear(inf, NUM_CLASSES)
model.cuda(GPU)

loc = 'cuda:{}'.format(GPU)
checkpoint = torch.load(saved_model_name, map_location=loc)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### 3.3 Make Uncertainty Estimates on the Data

In [ ]:
## new section to prep uncertainty estimates ##

transform_mri = transforms.Compose([
    transforms.Resize((imagesize,imagesize)),
    transforms.ToTensor(),
])

mri_dataset = datasets.ImageFolder(
    data_dir, transform=transform_mri)

# Make sure shuffle = False so that each monte carlo loop
# evaluates the same image at the same index
mri_loader = torch.utils.data.DataLoader(
        mri_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [ ]:
## new section to make input uncertainty estimates ## 
## make sure to run this section before and never after the model
## uncertainty, because model uncertainty adds dropout layers that 
## are permanently on

mri_predictions_input = get_input_monte_carlo_predictions(
    mri_loader, forward_passes = forward_passes,
    model = model, n_classes = NUM_CLASSES, n_samples = len(mri_dataset),
    num_channels = 3, image_size = imagesize
)

In [ ]:
## new section to make model uncertainty estimates ##

mri_predictions_model = get_model_monte_carlo_predictions(
    mri_loader, forward_passes = forward_passes,
    model = model, n_classes = NUM_CLASSES, n_samples = len(mri_dataset)
)

In [ ]:
## new section to process uncertainty estimates ##

# This object is a list to hold output uncertainties that come from the input data.
#
# mri_predictions_input is a tuple with both the mean predicted probabilities and 
# estimated uncertainties for all samples over each potential label. See function above.
#
# mri_predictions_input[0] accesses the means, mri_predictions_input[1] accesses the uncertainties
# mri_predictions_input[1][i] accesses uncertainties for the the i-th example over each output class / label
#
# Because this is binary classification, uncertainty in one label is equal to the uncertainty in the other. 
# i.e. mri_predictions_input[1][i][0] = mri_predictions_input[1][i][1], so only one is needed
mri_input_uncertainties = [mri_predictions_input[1][i][0] for i in range(len(mri_predictions_input[1]))]

# list to hold output uncertainties that come from the model
mri_model_uncertainties = [mri_predictions_model[1][i][0] for i in range(len(mri_predictions_model[1]))]

# calculate the combined uncertainty, which is just the sqrt of the sum of squares of each contribution
mri_combined_uncertainties = [((input_unc**2) + (model_unc**2))**0.5
                              for input_unc, model_unc 
                              in zip(mri_input_uncertainties,mri_model_uncertainties)]

In [ ]:
df = pd.DataFrame(mri_combined_uncertainties)
df.to_csv(out_file) 

In [ ]:
with open(out_file+'.pickle', 'wb') as f:
    pickle.dump(mri_combined_uncertainties, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
mri_combined_uncertainties[0:3]

In [ ]:
mri_input_uncertainties[0:3]

In [ ]:
mri_model_uncertainties[0:3]